# 环境设置

In [ ]:
import pandas as pd
import scanpy as sc
import numpy as np
import gc
from matplotlib.pyplot import rc_context

In [ ]:
sc.settings.set_figure_params(dpi=80, frameon=False, figsize=(4, 4), facecolor='white')

In [ ]:
ribo_genes = pd.read_table('ribo_genes.txt',skiprows=2, header=None)
ribo_genes

# 单样本测试

In [ ]:
adata = sc.read_csv('./rawdata_test/GSM5226576_C53ctr_raw_counts.csv.gz').T
adata.var_names_make_unique()
#adata.obs['Sample'] = path.split('_')[2] #'raw_counts/GSM5226574_C51ctr_raw_counts.csv'
sc.pp.filter_cells(adata, min_genes=100) #get rid of cells with fewer than 100 genes
sc.pp.filter_genes(adata, min_cells=3) #get rid of genes that are found in fewer than 3 cells
adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
adata.var['ribo'] = adata.var_names.isin(ribo_genes[0].values)
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribo'], percent_top=None, log1p=False, inplace=True)

upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, .98)
lower_lim = np.quantile(adata.obs.n_genes_by_counts.values, .02)
adata = adata[(adata.obs.n_genes_by_counts < upper_lim) & (adata.obs.n_genes_by_counts > lower_lim)]
adata = adata[adata.obs.pct_counts_mt < 20]
adata = adata[adata.obs.pct_counts_ribo < 2]
sc.pp.normalize_total(adata, target_sum=1e4) #normalize every cell to 10,000 UMI
sc.pp.log1p(adata) #change to log counts
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5) #these are default values
adata.raw = adata #save raw data before processing values and further filtering
adata = adata[:, adata.var.highly_variable] #filter highly variable
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt']) #Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed
sc.pp.scale(adata, max_value=10) #scale each gene to unit variance
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
adata.raw.shape

In [ ]:
sc.pl.pca(adata, color='CD4')

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=30, n_pcs=30)

In [ ]:
sc.tl.leiden(adata, resolution = 0.25)
sc.tl.umap(adata)

In [ ]:
sc.tl.paga(adata)
sc.pl.paga(adata, plot=True)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(adata, init_pos='paga')

In [ ]:
sc.pl.umap(adata, color=['leiden'],legend_loc='on data')

In [ ]:
del adata
gc.collect()

# 批量处理

In [ ]:
def pp(path):
    sc.settings.verbosity = 2  # a bit more logging
    adata = sc.read_csv(path).T
    adata.var_names_make_unique()
    adata.obs['Sample'] = path.split('_')[2] #'raw_counts/GSM5226574_C51ctr_raw_counts.csv'
    sc.pp.filter_cells(adata, min_genes=100) #get rid of cells with fewer than 100 genes
    sc.pp.filter_genes(adata, min_cells=3) #get rid of genes that are found in fewer than 3 cells
    adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
    adata.var['ribo'] = adata.var_names.isin(ribo_genes[0].values)
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribo'], percent_top=None, log1p=False, inplace=True)
    
    upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, .98)
    lower_lim = np.quantile(adata.obs.n_genes_by_counts.values, .02)
    adata = adata[(adata.obs.n_genes_by_counts < upper_lim) & (adata.obs.n_genes_by_counts > lower_lim)]
    adata = adata[adata.obs.pct_counts_mt < 20]
    adata = adata[adata.obs.pct_counts_ribo < 2]
    sc.pp.normalize_total(adata, target_sum=1e4) #normalize every cell to 10,000 UMI
    sc.pp.log1p(adata) #change to log counts
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5) #these are default values
    adata.raw = adata #save raw data before processing values and further filtering
    adata = adata[:, adata.var.highly_variable] #filter highly variable
    sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt']) #Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed
    sc.pp.scale(adata, max_value=10) #scale each gene to unit variance
    sc.tl.pca(adata, svd_solver='arpack')
    sc.pp.neighbors(adata, n_neighbors=10, n_pcs=20)
    sc.tl.leiden(adata, resolution = 0.25)
    sc.tl.umap(adata)
    return adata

In [ ]:
import os
out = []
for file in os.listdir('rawdata_test/'):
    out.append(pp('rawdata_test/' + file))
    
del file
gc.collect()

In [ ]:
# 取相同基因
var_names = []
for i in range(0,5):
    var_names = out[i].var_names.intersection(out[5].var_names)
    out[5] = out[5][:,var_names]
    print(i)

del i
gc.collect()

In [ ]:
for n in range(0,5):
    out[n] = out[n][:,var_names]
    print(n)
    
del n,var_names
gc.collect()

In [ ]:
sc.settings.verbosity = 2  # a bit more logging
for i, adata in enumerate(out[-5:]):
    print(f'... integrating batch {i}')
    #adata.obs['celltype_orig'] = adata.obs.celltype  # save the original cell type
    sc.tl.ingest(adata, out[0], obs='leiden')
    #adata.uns['leiden_colors'] = out1[5].uns['leiden_colors']  # fix colors

del i,adata
gc.collect()

In [ ]:
adata_ingest = out[0].copy()
for a in out[-5:]:
    adata_ingest = adata_ingest.concatenate(a)

del a
gc.collect()

In [ ]:
with rc_context({'figure.figsize': (4, 4)}):
    for s in out:
        sc.pl.umap(s, color = ['leiden','Sample'])

del s
gc.collect()

In [ ]:
sc.pl.umap(adata_ingest, color=['leiden','Sample'])

In [ ]:
adata_ingest.obs.groupby(['Sample', 'leiden']).count()

In [ ]:
del out
gc.collect()